In [ ]:
import pdb
import sys
import xlsxwriter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from math import log, exp
from random import randint
from PyNetSim import PyNetSim
from scipy.sparse import coo_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import precision_recall_curve,average_precision_score

# it's about to print thing in notebook, just ignore it ;)
np.set_printoptions(threshold=sys.maxint)
SIZE = 76
K = 100


class similarity_index:

    def __init__(k = 100, size, path):
        mataData = np.genfromtxt("../MSNs data/sigcomm2009.txt")
        size = size
        k = k


    """make a checkpoint for time slice

    :param step the step of each snapshot
    :param row each row of the mataData and it looks like
            node1, node2, linkstart, linkend, connect time, totally connected time

    """
    def time_slicer(steps, min_time, max_time):
        # how many snapshot we are going to have
        looper = (max_time - min_time)/steps
        checkpoints = [set([min_time + steps * l, min_time + steps * (l+1)]) for l in range(0, looper)]
        return checkpoints



    """
        Weight cacluation
    """
    def link_weight(duration):
        return  1 / (1 + exp(-float(duration)))


    "Generate spare martix"
    """
    One thing must remember is that node pair in the data has connected, so we don't need
    worried about take node pair without connected into account.
    """
    def weighted_spare_martix_generator(slice_start, slice_end):
        row = []
        col = []
        data = []
        k = K
        for i in mataData:
            if (i[2] >= slice_start and i[3] <= slice_end or
                i[2] <= slice_start and i[2] >= slice_start or
                i[3] <= slice_end and i[3] >= slice_end):

                if i[2] == i[3]:
                    duration = 1
                else:
                    duration = (i[3] if i[3] < slice_end else slice_end) - (i[2] if i[2] > slice_start else slice_start)

                # sum weight up if we already have that data
                if (i[0]-1) in row and col[row.index(int(i[0]-1))] == i[1]:
                    data[row.index(int(i[0]-1))] += link_weight(duration)
                else:
                    row.append(i[0]-1)
                    col.append(i[1]-1)
                    data.append(link_weight(duration, k))

        m = coo_matrix((data, (row, col)), shape=(SIZE, SIZE))
        return m


    def spare_martix_generator(slice_start, slice_end, mataData):
        row = []
        col = []
        data = []
        k = K
        for i in mataData:
            if (i[2] >= slice_start and i[3] <= slice_end or
                i[2] <= slice_start and i[2] >= slice_start or
                i[3] <= slice_end and i[3] >= slice_end):
                # pdb.set_trace()
                if i[0] in row and col[row.index(i[0])] == i[1]:
                    pass
                else:
                    row.append(i[0]-1)
                    col.append(i[1]-1)
                    data.append(1)
        m = coo_matrix((data, (row, col)), shape=(SIZE, SIZE))
        return m


    """"
        Caculate AUC
    """
    def calculate_auc(csrMatrix ,simScore, sample_time = 10000):
        slots = []
        samples = []
        score = []
        for i in range(0, sample_time):
            tempr = randint(0,SIZE-1)
            tempc = randint(0,SIZE-1)
            if tempc != tempr:
                slots.append([tempr, tempc])
        samples = np.array([csMatrix[slot[0], slot[1]] for slot in slots])
        score = np.array([simScore[slot[0], slot[1]] for slot in slots])

        return roc_auc_score(samples, score)


    """"
        plot roc
    """
    def calculate_roc(csrMatrix ,simScore, sample_time = 10000):
        slots = []
        samples = []
        score = []
        for i in range(0, sample_time):
            tempr = randint(0,SIZE-1)
            tempc = randint(0,SIZE-1)
            if tempc != tempr:
                slots.append([tempr, tempc])
        samples = np.array([csMatrix[slot[0], slot[1]] for slot in slots])
        score = np.array([simScore[slot[0], slot[1]] for slot in slots])

        return roc_curve(samples, score)


    """"
        Caculate pre recall
    """
    def calculate_pre_recall(csrMatrix ,simScore, sample_time = 10000):
        slots = []
        for i in range(0, sample_time):
            tempr = randint(0,SIZE-1)
            tempc = randint(0,SIZE-1)
            if tempc != tempr:
                slots.append([tempr, tempc])
        samples = np.array([csMatrix[slot[0], slot[1]] for slot in slots])
        score = np.array([simScore[slot[0], slot[1]] for slot in slots])

        return precision_recall_curve(samples, score), average_precision_score(samples, score)


    """
        Caculate cosin simiary
    """
    def calculate_cosin_simiary(adj_martix, wadj_martix):
        cosi_matrix = np.zeros((SIZE,SIZE))
        G=nx.from_numpy_matrix(adj_martix)
        for i in range(0, adj_martix.shape[0]):
            for j in range(i+1, adj_martix.shape[0]):
                # get a list cn of (i,j)
                # cnlist = sorted(nx.common_neighbors(G, i, j))
                cnlist = G.nodes()
                if len(cnlist) > 0:
                    listi = [[wadj_martix[i, c] for c in cnlist]]
                    listj = [[wadj_martix[j, c] for c in cnlist]]
                    cosim = cosine_similarity(listi, listj)
                    cosi_matrix[i,j] = cosim[0][0]
                    cosi_matrix[j,i] = cosim[0][0]
        return cosi_matrix


    """
        Caculate cosin simiary with IDF
    """
    def calculate_cosin_idf_simiary(adj_martix, wadj_martix):
        cosi_matrix = np.zeros((SIZE,SIZE))
        G=nx.from_numpy_matrix(adj_martix)
        for i in range(0, adj_martix.shape[0]):
            for j in range(i+1, adj_martix.shape[0]):
                # get a list cn of (i,j)
                # cnlist = sorted(nx.common_neighbors(G, i, j))
                cnlist = G.nodes()
                if len(cnlist) > 0:
                    listi = [[wadj_martix[i, c] for c in cnlist]]
                    listj = [[wadj_martix[j, c] for c in cnlist]]
                    cosim = cosine_similarity(listi, listj)
                    cosi_matrix[i,j] = cosim[0][0]
        return cosi_matrix